In [3]:
import networkx as nx

In [123]:
def addNode(G, node):
    i,j = node
    G.add_node(i+j*7)
    return G

def addPlayer(G, src, dest, player):
    i_src, j_src = src # nodo de origen (donde se hace el primer clic)
    i_dest, j_dest = dest # nodo de destino (donde se hace el segundo clic)
    src_pos = i_src*7+j_src # posición del origen en notación vectorial (no matricial)
    dest_pos = i_dest*7+j_dest # posición del destino en notación vectorial (no matricial)
    G.nodes[src_pos]['player'] = player # el nodo del origen se queda pintado en el color del player
    G.nodes[dest_pos]['player'] = 1 if player==2 else 2 # el nodo de destino se queda pintado en el color del rival
    G.remove_edge(src_pos, dest_pos, key = 'possible_movement') # se elimina la arista src_dest como posible mov.
    G = updatePlayers(G, src_pos, src, player)
    G.remove_edges_from([edge for edge in G.edges(dest_pos) if G.has_edge(*edge, key = 'possible_movement')])
    G = addEdges(G, *dest)
    return G

def addFirstPlayer(G, dest, player):
    i,j = dest
    G.nodes[i+j*7]['player'] = player
    G = addEdges(G, *dest)
    return G

def addEdges(G, i, j):
    possible_movements = {(i+1,j),(i-1,j),(i,j+1),(i,j-1),(i+1,j-1),(i-1,j+1)}
    possible_movements = [(i*7+j, movement[0]*7 + movement[1]) for movement in possible_movements
                          if (movement in [(i,j) for i in range(7) for j in range(7)] # si la posición existe en el tablero
                               and not G.nodes[movement[0]*7+movement[1]]['player'])] # si no hay fichas en esa casilla
    [G.add_edge(i,j, key = 'possible_movement') for (i,j) in possible_movements]
    return G

def updatePlayers(G, src_pos, src, player):
    i,j = src
    possible_movements = {(i+1,j),(i-1,j),(i,j+1),(i,j-1),(i+1,j-1),(i-1,j+1)}
    possible_movements = [(i*7+j, movement[0]*7 + movement[1]) for movement in possible_movements
                          if (movement in [(i,j) for i in range(7) for j in range(7)] # si la posición existe en el tablero
                               and G.nodes[movement[0]*7+movement[1]]['player'] == player)]
    G.remove_edges_from([edge for edge in G.edges(src_pos) if not G.has_edge(*edge, key = 'possible_movement')])
    [G.add_edge(i,j, key = 'path') for (i,j) in possible_movements]
    return G

def getPossibleMovements(G, player):
    return [edge for node in G.nodes for edge in G.edges(node)
            if G.nodes[node]['player'] == (1 if player==2 else 2)]
    
def getBoard(G):
    return [[G.nodes[i*7+j]['player'] for j in range(7)] for i in range(7)]

In [124]:
G = nx.MultiGraph()
G.add_nodes_from(range(49), player = 0)

In [125]:
G = addFirstPlayer(G, (1,1), 2)

In [126]:
G.edges

MultiEdgeView([(1, 8, 'possible_movement'), (2, 8, 'possible_movement'), (7, 8, 'possible_movement'), (8, 9, 'possible_movement'), (8, 15, 'possible_movement'), (8, 14, 'possible_movement')])

In [127]:
G = addPlayer(G, (1,1), (1,2), 1)

In [128]:
G.edges

MultiEdgeView([(1, 8, 'possible_movement'), (2, 8, 'possible_movement'), (2, 9, 'possible_movement'), (3, 9, 'possible_movement'), (7, 8, 'possible_movement'), (8, 15, 'possible_movement'), (8, 14, 'possible_movement'), (9, 10, 'possible_movement'), (9, 15, 'possible_movement'), (9, 16, 'possible_movement')])

In [129]:
G = addPlayer(G, (1,1), (2,1), 2)

In [130]:
G.edges

MultiEdgeView([(1, 8, 'possible_movement'), (2, 8, 'possible_movement'), (2, 9, 'possible_movement'), (3, 9, 'possible_movement'), (7, 8, 'possible_movement'), (8, 14, 'possible_movement'), (8, 9, 'path'), (9, 10, 'possible_movement'), (9, 16, 'possible_movement'), (14, 15, 'possible_movement'), (15, 21, 'possible_movement'), (15, 22, 'possible_movement'), (15, 16, 'possible_movement')])